# True Value Testing

In [1]:
import numpy as np
import tvm
from tvm import te
import d2ltvm

successd...



In [22]:
shape = (3,4)
padding = 1
dtype = 'float32'
a = np.ones(shape=shape, dtype=dtype)
b = np.zeros([i + 2*padding for i in shape], dtype=dtype)
b[padding:-padding, padding:-padding] = a
b

array([[0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

In [31]:
n, m = te.var(name='n'), te.var(name='m')
p = te.var(name='p')
A = te.placeholder((n, m), dtype=dtype, name='a')
B = te.compute((n + 2 * p, m + 2 * p), lambda i, j: te.if_then_else(
        te.any(i < p, i >= n + p, j < p, j >= m + p), 0.0, A[i - p, j - p]))

s = te.create_schedule(B.op)
tvm.lower(s, [A, B]).show()

In [32]:
mod = tvm.build(s, [A, B, n, m, p])
c = tvm.nd.array(np.empty([i + 2 * padding for i in shape], dtype=dtype))
mod(tvm.nd.array(a), c, *shape, padding)
np.testing.assert_equal(b, c.asnumpy())

# Summary
1.We can use tvm.any and tvm.all to construct complex conditional expressions